In [17]:
import nltk.tokenize
from nltk.grammar import DependencyGrammar
from nltk.parse import (
    DependencyGraph,
    ProjectiveDependencyParser,
    NonprojectiveDependencyParser,
)
from collections import Counter, defaultdict

In [2]:
trees = open('trees_conll.txt').read().split('\n\n')

In [3]:
dp = DependencyGraph(trees[2])

In [4]:
list(dp.triples())

[((4, 'be', 'VERB'), 'prep', (1, 'beneath', 'ADP')),
 ((1, 'beneath', 'ADP'), 'pobj', (3, 'mantle', 'NOUN')),
 ((3, 'mantle', 'NOUN'), 'det', (2, 'the', 'DET')),
 ((4, 'be', 'VERB'), 'attr', (8, 'appendage', 'NOUN')),
 ((8, 'appendage', 'NOUN'), 'nummod', (5, 'three', 'NUM')),
 ((8, 'appendage', 'NOUN'), 'amod', (6, 'major', 'ADJ')),
 ((8, 'appendage', 'NOUN'), 'amod', (7, 'cortical', 'ADJ')),
 ((8, 'appendage', 'NOUN'), 'punct', (9, ':', 'PUNCT')),
 ((8, 'appendage', 'NOUN'), 'appos', (12, 'ganglion', 'NOUN')),
 ((12, 'ganglion', 'NOUN'), 'det', (10, 'the', 'DET')),
 ((12, 'ganglion', 'NOUN'), 'amod', (11, 'basal', 'NOUN')),
 ((12, 'ganglion', 'NOUN'), 'cc', (13, 'and', 'CCONJ')),
 ((12, 'ganglion', 'NOUN'), 'conj', (14, 'cerebellum', 'NOUN')),
 ((12, 'ganglion', 'NOUN'), 'punct', (15, '(', 'PUNCT')),
 ((12, 'ganglion', 'NOUN'), 'relcl', (19, 'regulate', 'VERB')),
 ((19, 'regulate', 'VERB'), 'nsubj', (16, 'both', 'DET')),
 ((16, 'both', 'DET'), 'prep', (17, 'of', 'ADP')),
 ((17, 'of',

In [8]:
?dp.triples()

In [30]:
def noun_noun(triples):
    rels = defaultdict(set)
    for head, rel, dep in triples:
        if head[2] == 'NOUN' and dep[2] == 'NOUN':
            h, d = sorted([head, dep], key=lambda x: x[0])
            rels[rel].update([(h[1], d[1])])
    return rels


In [31]:
c = defaultdict(set)
for tree in trees:
    try:
        rels = noun_noun(DependencyGraph(tree).triples())
        for key in rels:
            c[key].update(rels[key])
    except (ValueError, KeyError, AssertionError, TypeError):
        continue

In [19]:
d = defaultdict(list)

In [25]:
b = defaultdict(list)

In [ ]:
c = b.copy()

In [26]:
c.update(d)

TypeError: can only concatenate list (not "int") to list

In [33]:
c.keys()

dict_keys(['attr', 'pcomp', 'mark', 'intj', 'nsubjpass', 'case', 'appos', 'ccomp', 'acomp', 'acl', 'neg', 'csubj', 'advmod', 'oprd', 'cc', 'preconj', 'advcl', 'nsubj', 'compound', 'predet', 'det', 'xcomp', 'punct', 'nmod', 'poss', 'auxpass', 'meta', 'aux', 'amod', 'expl', 'quantmod', 'prt', 'dobj', 'dative', 'dep', 'conj', 'relcl', 'npadvmod', 'parataxis', 'nummod', 'prep', 'pobj'])

In [40]:
c['appos']

{('rule', 'ethnography'),
 ('action', 'action'),
 ('spécialité', 'systématisation'),
 ('response', 'teacher'),
 ('sens', 'institution'),
 ('pp', 'station'),
 ('approximant', 'ɹ'),
 ('defénce', 'stress'),
 ('likelihood', 'training'),
 ('probe', 'aspect'),
 ('unit', 'arrangement'),
 ('a', 'father'),
 ('mouth', 'metonymy'),
 ('permutation', 'stratum'),
 ('change', 'effect'),
 ('pronoun', 'sign'),
 ('knowledge', 'distinction'),
 ('something', 'a'),
 ('assumption', 'ﬁeld'),
 ('asymmetry', 'language'),
 ('criterion', 'presence'),
 ('чeлов', 'ко́смос'),
 ('chunk', 'regent'),
 ('morpheme', 'school'),
 ('representation', 'context'),
 ('assessment', 'principle'),
 ('cat', 'perfective'),
 ('xiv', 'xviii'),
 ('seʔ', 'ʔeseʔ'),
 ('inanimate', 'possession'),
 ('shortening', 'a.'),
 ('question', 'order'),
 ('symbol', 'stop'),
 ('calle', 'hora'),
 ('car.’', 'dele'),
 ('genitive', '19e'),
 ('specification', 'vowel'),
 ('awareness', 'effect'),
 ('më', 'janar'),
 ('knowledge', 'ev.per'),
 ('compoundâ•‡',

In [1]:
import requests
from lxml import html
import os
from time import sleep

In [61]:
r = requests.get('https://cyberleninka.ru/article/n/paket-bibliotek-dlya-avtomatizatsii-elektrotehnicheskih-otdelov-rubius-electric-suite')

In [62]:
tree = html.fromstring(r.text)

In [63]:
tree.xpath('//i[@itemprop="keywords"]/span/text()')

['ЛЭП',
 'МЗ',
 'линия электропередачи',
 'система молниезащиты',
 'AutoCAD',
 'КОМПАС',
 'выходная документация']

In [4]:
cbl = 'https://cyberleninka.ru'

In [6]:
os.chdir('/home/mnefedov/hdd/cyberleninka')
r = requests.get(cbl+'/article')
tree = html.fromstring(r.text)
topics = tree.xpath('//ul[@class="grnti"]/li/a')
for topic in set(topics):
    name = topic.text[:40]
    try:
        if len(os.listdir(name.replace(' ', '_'))) > 50:
            print('Skipping _ ', name)
            continue
    except FileNotFoundError:
        pass
    url = topic.xpath('@href')[0]
    try:
        os.mkdir(name.replace(' ', '_'))
    except FileExistsError:
        pass
    links = []
    for i in range(1, 20):
        req = requests.get(cbl + url + '/{}'.format(i))
        tree = html.fromstring(req.text)
        ls = tree.xpath('//ul[@class="list"]/li/a/@href')
        links += ls
        sleep(1)
    print('GOT - {} links for topic - {}'.format(len(links), name))
    if not links:
        print('No links for ', name)
        continue
    
    for j, link in enumerate(links):
        if link.split('/')[-1] + '.art.pdf' in os.listdir(name.replace(' ', '_')):
            print('skip')
            continue
        req = requests.get(cbl + link)
        if 'Вы точно человек?' in req.text:
            print('Capcha')
            inp = input('Пройди капчу на кб и продолжи...')
            if inp == 'Y':
                continue
        tree = html.fromstring(req.text)
        kws = tree.xpath('//i[@itemprop="keywords"]/span/text()')
        if kws:
            try:
                pdf = requests.get(cbl + link + '.pdf')
            except Exception as e:
                print('ERROR', e)
                print(cbl + link + '.pdf')
                continue
            if pdf.status_code == 200:
                f = open(name.replace(' ', '_') + '/' + link.split('/')[-1] + '.art.pdf', 'wb')
                f.write(pdf.content)
                f.close()
                f = open(name.replace(' ', '_') + '/' + link.split('/')[-1] + '.kws.txt', 'w')
                f.write('\n'.join(kws))
                f.close()
            else:
                print('Could not get pdf!')
        if not j % 10:
            print('Processed - ', j)
        sleep(1)
                
    

Skipping _  Педагогика
Skipping _  Пищевая промышленность
Skipping _  Полиграфия
Skipping _  Политика и политические науки
Skipping _  Приборостроение
GOT - 7 links for topic - Прочие отрасли экономики
skip
skip
skip
skip
skip
skip
Skipping _  Массовая коммуникация
Skipping _  Психология
Skipping _  Радиотехника
GOT - 14 links for topic - Изобретательство
Processed -  0
skip
skip
skip
skip
skip
Processed -  10
skip
skip
skip
GOT - 14 links for topic - Рационализаторство
Processed -  0
skip
skip
skip
skip
skip
Processed -  10
skip
skip
skip
Skipping _  Информатика
Skipping _  Религия
Skipping _  Репрография
Skipping _  Рыбное хозяйство
Skipping _  Журналистика
Skipping _  Связь
Skipping _  Сельское и лесное хозяйство
Skipping _  Социология
Skipping _  Комплексное изучение отдельных стран и р
Skipping _  Средства массовой информации
GOT - 18 links for topic - Стандартизация
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
GOT - 24 links for topic - Статисти

In [159]:
from collections import deque, defaultdict

In [292]:
rt = 'https://russian.rt.com'
# found = 0
# r = requests.get(rt + '/search?q=закон&type=&df=2015-01-20&dt=')
# tree = html.fromstring(r.text)
# links = [x for x in tree.xpath('//@href') if 'article/' in x]
deq = deque(arts)
# art_dict = defaultdict(lambda: {})
# js = 
# already = set()
while deq:
#     sleep(1)
    link = deq.popleft().split('#')[0]
    num = re.findall('./article/(\d+)', link)
    if not num or num[0] in already:
        continue
    already.add(num[0])
    
    try:
        if link.startswith('http'):
            if 'russian.rt' in link:
                
                r = requests.get(link, timeout=2)
            else:
                continue
        else:
            r = requests.get(rt + link, timeout=2)
    except Exception as e:
        print(e)
        continue
    
    if r.status_code == 200:
        try:
            tre = html.fromstring(r.text)
        except Exception as e:
            print(e)
            continue
    else:
        print('not 200!')
        continue
        
    new_links = [x for x in tre.xpath('//@href') if 'article/' in x]
    deq.extend(new_links)
    kws = [re.sub('\n *', '',  x.lower()) 
           for x in tre.xpath('//a[@class="tags-trends__link link link_underline_color"]/text()')]
    
    if kws:
        title = tre.xpath('//h1[@class="article__heading article__heading_article-page"]/text()')
        if title:
            title = title[0]
        else:
            continue
        
        summary = tre.xpath('//div[@class="article__summary article__summary_article-page article__summary_article-page_note js-mediator-article"]/text()')
        summary = ' '.join(summary)
        
        content = []
        raw = tre.xpath('//div[@class="article__text article__text_article-page js-mediator-article"]//text()')
        if not raw:
            continue
        for line in raw:
            line = re.sub('(\n +)|(\xa0)', ' ', line)
            if len(set(line)) > 1:
                content.append(line)
        
        
        art_dict[link]['title'] = title
        art_dict[link]['summary'] = summary
        art_dict[link]['content'] = content
        art_dict[link]['keywords'] = kws
        found += 1
        if found > 10000:
            break
        
        if not found % 500:
            print('Found - ', found)
        
        

In [313]:
s = set()
for k in art_dict:
    if k.startswith('/'):
        k = rt + k
    s.add(k)

In [314]:
len(s)

7218

In [315]:
len(art_dict)

7218

In [286]:
[x for x in tre.xpath('//@href')]

['/favicon.ico',
 '/favicon.ico',
 '/static/blocks/touch-icon/apple-touch-icon-precomposed.png',
 '/static/blocks/touch-icon/apple-touch-icon-72x72-precomposed.png',
 '/static/blocks/touch-icon/apple-touch-icon-114x114-precomposed.png',
 '/static/blocks/touch-icon/apple-touch-icon-144x144-precomposed.png',
 'https://russian.rt.com/nopolitics/article/383642-genocid-armyan-film-premiera/amp',
 '/static/css/template.css?v=4ffa2b77fa5e6875a15937f887312374c79712371676d7576a675467f5767767e76763777d7870c0714',
 '/static/css/label.css?v=1',
 '//vk.com/rt_russian',
 '//twitter.com/RT_russian',
 '//www.facebook.com/RTRussian',
 '//telegram.me/joinchat/BKqxYj3FocDllxZxKyqNow',
 '//www.youtube.com/user/rtrussian',
 '//ok.ru/rtrussian',
 '//plus.google.com/+rtrussian',
 '//rt-russian.livejournal.com/',
 '//www.instagram.com/rtrussian/',
 '//chats.viber.com/rtrussian',
 '/rss',
 '//www.rt.com/',
 '//deutsch.rt.com/',
 '//francais.rt.com/',
 '//arabic.rt.com/',
 '//actualidad.rt.com/',
 '//doc.rt.com

In [200]:
import pickle
import re

In [294]:
js = json.dumps(dict(art_dict))

In [295]:
f = open('rt_full.json', 'w')
f.write(js)
f.close()

In [223]:
f = open('rt_dict_6k.txt', 'w')
f.write(js)
f.close()

In [225]:
jss = json.loads(open('rt_dict_6k.txt', 'r').read())

In [227]:
# jss

In [201]:
_ = pickle.dump(dict(art_dict), open('rt_dict_6k.txt', 'wb'))

In [330]:
ng = 'http://www.ng.ru'
r = requests.get(ng)
tree = html.fromstring(r.text)
links = [x for x in tree.xpath('//@href') 
         if (x.endswith('html') or x.endswith('/')) 
         and ('photo' not in x and 'video' not in x)]
deq = deque(links)
already = set()
dict_ng = defaultdict(lambda: {})
found = 0
while deq:
    link = deq.popleft().split('#')[0]
    
    if 'http' in link and '.ng.ru' not in link:
        continue
    if link.startswith('/'):
        if link.startswith('/http'):
            pass
        else:
            link = ng + link
    
    if link in already:
        continue
    

    
    already.add(link)
    try:
        r = requests.get(link, timeout=2)

    except Exception as e:
        print(e)
        continue
    
    if r.status_code == 200:
        try:
            tre = html.fromstring(r.text)
        except Exception as e:
            print(e)
            continue
    else:
        print('not 200!')
        print(link)
        continue
    
    new_links = [x for x in tre.xpath('//@href') 
         if (x.endswith('html') or x.endswith('/')) 
         and ('photo' not in x and 'video' not in x)]
    deq.extend(new_links)
    
    kws = tre.xpath('//p[@class="tags"]/a/text()')
    if not kws:
        continue
    
    content = [x for x in tre.xpath('//article[@class="typical"]/p/text()') if len(set(x)) > 3]
    if not content:
        continue
    
    title = ''.join(tre.xpath('//h1[@class="htitle black"]/text()'))
    if not title:
        continue
    
    subt = ''.join(tre.xpath('//p[@class="anonce"]/text()'))
    
    dict_ng[link]['keywords'] = kws
    dict_ng[link]['content'] = content
    dict_ng[link]['title'] = title
    dict_ng[link]['sub'] = subt
    
    found += 1
    
    if found > 3000:
        break
    if not found % 50:
        print(found)

HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
50
not 200!
http://www.ng.ru/authors/95145/
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
not 200!
http://www.ng.ru/authors/4140/
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
not 200!
http://www.ng.ru/forum/
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
HTTPConnectionPool(host='www.ng.ru', port=80): Read timed out. (read timeout=2)
100

not 200!
http://www.ng.ru/auth/?forgot_password=yes&backurl=%2Fworld%2F2018-03-22%2F1_7195_usa.html
No connection adapters were found for '/http://www.ng.ru/world/2018-03-21/7_7195_london.html'
No connection adapters were found for '/http://www.ng.ru/politics/2018-03-21/3_7195_vilnus.html'
No connection adapters were found for '/http://www.ng.ru/economics/2018-03-21/100_finance210318.html'
not 200!
http://www.ng.ru/authors/9280/
No connection adapters were found for '/http://www.ng.ru/cis/2018-03-21/100_obzor210318.html'
not 200!
http://www.ng.ru/auth/?forgot_password=yes&backurl=%2Feconomics%2F2018-03-22%2F1_7195_gasprom.html
not 200!
http://www.ng.ru/ng_energiya/2018-02-13/9_7171_potok.html
No connection adapters were found for '/http://www.ng.ru/economics/2018-03-21/1_7194_budapesht.html'
not 200!
http://www.ng.ru/economics/2018-03-20/100_obzor200318.html
No connection adapters were found for '/http://www.ng.ru/economics/2018-03-20/100_obzor200318.html'
not 200!
http://www.ng.ru/aut

not 200!
http://www.ng.ru/people/2018-03-07/11_438_polska.html
No connection adapters were found for '/http://www.ng.ru/people/2018-03-07/11_438_polska.html'
not 200!
http://www.ng.ru/people/2018-02-21/12_437_papa.html
No connection adapters were found for '/http://www.ng.ru/people/2018-02-21/12_437_papa.html'
not 200!
http://www.ng.ru/auth/?forgot_password=yes&backurl=%2Fworld%2F2018-03-20%2F1_7193_trump.html
not 200!
http://www.ng.ru/energy/2018-03-20/15_7193_siria.html
not 200!
http://www.ng.ru/world/2018-03-20/7_7193_turkey.html
not 200!
http://www.ng.ru/authors/4208/
No connection adapters were found for '/http://www.ng.ru/world/2018-03-21/2_7194_germany.html'
No connection adapters were found for '/http://www.ng.ru/kartblansh/2018-03-21/3_7194_kartblansh.html'
not 200!
http://www.ng.ru/auth/?forgot_password=yes&backurl=%2Farmies%2F2018-03-18%2F100_usasyria1803.html
not 200!
http://www.ng.ru/authors/13640/
not 200!
http://nvo.ng.ru/wars/2018-02-22/1_985_siria.html
not 200!
http://

not 200!
http://www.ng.ru/politics/2018-02-01/2_7163_opposition.html
not 200!
http://www.ng.ru/cis/2018-01-31/6_7163_donbass.html
not 200!
http://www.ng.ru/world/2018-01-31/6_7162_normand.html
not 200!
http://www.ng.ru/cinematograph/2018-02-24/100_ber2002.html
not 200!
http://www.ng.ru/cinematograph/2018-02-22/100_romaniafilms.html
not 200!
http://www.ng.ru/cinematograph/2018-02-20/100_7177_berlinale.html
not 200!
http://www.ng.ru/cinematograph/2018-02-18/100_7175_berlinale.html
not 200!
http://www.ng.ru/cinematograph/2018-02-17/100_170218_berlinale.html
not 200!
http://www.ng.ru/cinematograph/2018-02-16/100_berlin18_02.html
not 200!
http://www.ng.ru/cinematograph/2018-02-15/100_7174_berlinale.html
not 200!
http://www.ng.ru/cis/2018-03-20/5_7194_ukraina.html
not 200!
http://www.ng.ru/cis/2018-03-20/1_7193_pridnestrovie.html
not 200!
http://www.ng.ru/politics/2018-03-19/1_7193_west.html
not 200!
http://www.ng.ru/politics/2018-03-19/100_obzor190318.html
not 200!
http://www.ng.ru/cis/2018

not 200!
http://www.ng.ru/cis/2018-03-14/5_7189_kirgisia.html
No connection adapters were found for '/http://www.ng.ru/cis/2018-03-14/5_7189_kirgisia.html'
not 200!
http://www.ng.ru/auth/?forgot_password=yes&backurl=%2Fkoncep%2F2017-10-19%2F13_908_life.html
not 200!
http://www.ng.ru/koncep/
not 200!
http://www.ng.ru/authors/71864/
not 200!
http://www.ng.ru/politic/
not 200!
http://www.ng.ru/rakurs_rel/
not 200!
http://www.ng.ru/people/
not 200!
http://www.ng.ru/printing/
not 200!
http://www.ng.ru/bogoslovie/
not 200!
http://www.ng.ru/society/
not 200!
http://www.ng.ru/problems/2018-03-21/14_439_infidels.html
not 200!
http://www.ng.ru/faith/2018-03-13/100_otpevanie.html
not 200!
http://www.ng.ru/faith/2018-03-13/2_7188_germany.html
not 200!
http://www.ng.ru/faith/2018-03-01/100_music010318.html
not 200!
http://www.ng.ru/faith/2018-03-01/1_7182_kokain.html
not 200!
http://www.ng.ru/faith/2018-02-27/100_drugscandal.html
not 200!
http://www.ng.ru/faith/2018-02-26/2_7180_grob.html
not 200!


...

not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2018-03-16/16_988_crocodile.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2017-12-22/16_978_fireworks.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2017-11-03/16_972_bear.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2017-09-29/16_967_altona.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2017-08-11/14_960_afalina.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2017-04-07/16_943_bereg.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2017-03-10/16_939_varenie.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2014-11-21/16_wizard.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2014-10-24/16_smi.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://nvo.ng.ru/notes/2014-09-12/16_jacket.html
not 200!
http://nvo.

not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_exlibris/2018-03-22/9_927_gorky.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/authors/10343/
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/non-fiction/2018-03-01/15_925_nobelevka.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/non-fiction/2018-03-01/15_925_krasnov.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/fakty/2018-02-08/11_922_nos.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/non-fiction/2018-02-01/15_921_pobedonoscev.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_against.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/zavisimaya/2017-12-19/14_7139_chapaev.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_exlibris/2018-03-22/16_927_santa.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/authors/34607/
not 

not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_politics/2017-04-18/9_6976_nenashi.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_politics/2017-04-18/9_6976_children.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/authors/36332/
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_politics/2017-04-18/11_6976_ldpr.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/authors/5666/
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_politics/2017-04-18/11_6976_future.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_politics/2017-04-18/11_6976_force.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/authors/81825/
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/authors/26275/
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www

not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/five/2018-02-08/9_922_5books.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/five/2018-02-01/9_921_5books.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/five/2018-01-25/9_920_5books.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/five/2018-01-18/9_919_5books.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/five/2018-01-11/9_918_5books.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/five/2017-12-21/9_917_5book.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/five/2017-12-14/9_916_5books.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/fakty/2018-03-01/11_925_sharf.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/fakty/2018-03-01/11_925_life.html
not 200!
http://www.ng.ru/mobile/ng-ru/amp/?p=http://www.ng.ru/fakty/2018-03-01/11_925_kabush.html
not 200!
http://www.ng.ru/mo

KeyboardInterrupt: 

In [331]:
found

370

In [323]:
r = requests.get(link, timeout=2)

In [324]:
r.status_code

200